# Import Libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from scipy.sparse import csr_matrix
# from gensim.models import Word2Vec
# import joblib

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.constraints import max_norm
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [ ]:
# !pip install transformers

# Load Data


In [ ]:
df = pd.read_csv('mental_health.csv')
df.head()
df.shape

#Data Preprocessing




In [ ]:
# Check data types
df.dtypes

In [ ]:
# Check missing values
df.isnull().sum()

The dataset does not contain any missing value.

In [ ]:
# Check the distribution of mental health status
label_0_count = df[df['label']==0].shape[0]
label_1_count = df[df['label']==1].shape[0]
print("Number of entries with label = 0: {}".format(label_0_count))
print("Number of entries with label = 1: {}".format(label_1_count))

# Experimenting with Data

## Vectorize "text" variable

In [ ]:
# Vectorize "text" using CountVectorizer
# Documentation 1: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# Documentation 2: https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

vectorizer = CountVectorizer()
X = vectorizer.fit(df['text'])
print("Vocabulary: ", vectorizer.vocabulary_)

features = vectorizer.transform(df['text'])
features = features.toarray()

In [ ]:
print(features.shape)

In [ ]:
# Split data into training set and testing set
x_train, x_test, y_train, y_test = train_test_split(features, df['label'], train_size=0.8, random_state=0)
print("Number of entries in the training set: {}".format(x_train.shape[0]))
print("Number of entries in the testing set: {}".format(x_test.shape[0]))

In [ ]:
# Get the vocabulary from the CountVectorizer object
vocabulary = vectorizer.get_feature_names_out()

# Sum the counts of each word across all documents
word_counts = np.sum(features, axis=0)

# Sort the words in descending order of frequency
sorted_idx = np.argsort(word_counts)[::-1]

# Get the top 20 words by frequency
top_words = [vocabulary[i] for i in sorted_idx[:20]]
top_counts = [word_counts[i] for i in sorted_idx[:20]]

# Plot the frequency of the top 20 words
plt.barh(top_words, top_counts)
plt.title('Frequency of Top 20 Words')
plt.xlabel('Frequency')
plt.ylabel('Word')
plt.show()

In [ ]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
# csr_matrix compress sparse matrix
x_train_csr = csr_matrix(x_train)
x_test_csr = csr_matrix(x_test)

In [ ]:
reg = LogisticRegression(max_iter=1000)
reg.fit(x_train_csr, y_train)

# predict on test data
y_pred = reg.predict(x_test_csr)

In [ ]:
accuracy_score(y_pred, y_test)

## Feature selection with PCA

In [ ]:
# Define the number of components to keep
n_components = 1000

# Perform PCA feature reduction
pca = PCA(n_components=n_components)
x_train_selected = pca.fit_transform(x_train)
x_test_selected = pca.transform(x_test)

In [ ]:
# Save the PCA model and transformed data
joblib.dump(pca, 'pca_model.pkl')
np.save('x_train_pca.npy', x_train_selected)
np.save('x_test_pca.npy', x_test_selected)

In [ ]:
# Load the saved PCA model
pca = joblib.load('pca_model.pkl')

# Load the saved transformed data
x_train_selected = np.load('x_train_pca.npy')
x_test_selected = np.load('x_test_pca.npy')

In [ ]:
reg = LogisticRegression(max_iter=1000)
reg.fit(x_train_selected, y_train)

# predict on test data
y_pred = reg.predict(x_test_selected)

In [ ]:
accuracy_score(y_pred, y_test)

## Random Forest

In [ ]:
# Define the parameter grid for the random forest classifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [100, 200, 300],
    'min_samples_split': [50, 100, 200, 300]
}

# Create a random forest classifier object
rfc = RandomForestClassifier()

# Create a grid search object with cross-validation
grid_search = GridSearchCV(rfc, param_grid, cv=5)

# Fit the grid search object on the training data
grid_search.fit(x_train_selected, y_train)

# Print the best hyperparameters found by the grid search
print("Best hyperparameters:", grid_search.best_params_)

In [ ]:
# Use the best hyperparameters to train a random forest classifier on the training data
best_rfc = RandomForestClassifier(**grid_search.best_params_)
best_rfc.fit(x_train_selected, y_train)

# Use the classifier to make predictions on the testing data
predictions = best_rfc.predict(x_test_selected)

# Evaluate the best classifier on the testing data
accuracy = best_rfc.score(x_test_selected, y_test)
print("Accuracy:", accuracy)

# Print the confusion matrix
cm = confusion_matrix(y_test, predictions)
print('Confusion matrix:\n', cm)

## Neural Networks

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(x_train_selected.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# change learning rate
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.001),
              metrics=['accuracy'])

# train model
model.fit(x_train_selected, y_train, epochs=50, batch_size=32,
          validation_data=(x_test_selected, y_test))

# evaluate model on test set
test_loss, test_acc = model.evaluate(x_test_selected, y_test)
print('Test accuracy:', test_acc)

## Random Froest with Word2Vec

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=0)

# Train a Word2vec model on the training data
corpus = [doc.split() for doc in train_data]
model = Word2Vec(corpus, vector_size=100, window=5, min_count=5, workers=4)

In [ ]:
# Define a function to generate feature vectors for each comment
def get_vector(text):
    words = text.split()
    vec = np.zeros((100,))
    count = 0
    for word in words:
        if word in model.wv.key_to_index:
            vec += model.wv[word]
            count += 1
    if count > 0:
        return vec / count
    else:
        return vec

In [ ]:
# Generate feature vectors for the training and testing data
train_features = np.vstack([get_vector(text) for text in train_data])
test_features = np.vstack([get_vector(text) for text in test_data])

# Train a random forest classifier on the training data
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_features, train_labels)

# Use the classifier to make predictions on the testing data
predictions = rfc.predict(test_features)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(test_labels, predictions)
print('Accuracy:', accuracy)

# Print the confusion matrix
cm = confusion_matrix(test_labels, predictions)
print('Confusion matrix:\n', cm)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(train_features.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# change learning rate
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.001),
              metrics=['accuracy'])

# train model
model.fit(train_features, train_labels, epochs=50, batch_size=32,
          validation_data=(test_features, y_test))

# evaluate model on test set
test_loss, test_acc = model.evaluate(test_features, y_test)
print('Test accuracy:', test_acc)

## BERT with NN

In [ ]:
# BERT Embedding
'''
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import TFBertModel
import tensorflow as tf
from tensorflow import keras
from transformers import TFDistilBertModel

def main():
    # Load the dataset
    df = pd.read_csv('mental_health.csv')
    df = df.iloc[:1500]

    # Split the dataset into training and testing sets
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Initialize the BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenize the text data
    train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
    test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True)

    # Perform feature selection
    selector = SelectKBest(chi2, k='all')
    train_features = np.array(train_encodings['input_ids'])
    train_features = selector.fit_transform(train_features, train_df['label'])
    train_attention_mask = np.array(train_encodings['attention_mask'])
    train_attention_mask = selector.transform(train_attention_mask)
    test_features = np.array(test_encodings['input_ids'])
    test_features = selector.transform(test_features)
    test_attention_mask = np.array(test_encodings['attention_mask'])
    test_attention_mask = selector.transform(test_attention_mask)

    # Load the BERT model
    #bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    # Generate the BERT embeddings
    train_embeddings = bert_model([train_features, train_attention_mask])[0][:, 0, :]
    test_embeddings = bert_model([test_features, test_attention_mask])[0][:, 0, :]

    # Save the train and test embeddings to files
    np.save('train_embeddings.npy', train_embeddings)
    np.save('test_embeddings.npy', test_embeddings)

    # Load the saved embeddings from files
    #train_embeddings = np.load('train_embeddings.npy')
    #test_embeddings = np.load('test_embeddings.npy')


    # Define the neural network model
    model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(768,)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the neural network model
    model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
                metrics=['accuracy'])

    # Train the neural network model
    model.fit(train_embeddings, train_df['label'], epochs=100, batch_size=32)

    # Evaluate the neural network model on the test data
    test_loss, test_acc = model.evaluate(test_embeddings, test_df['label'])
    print('Test accuracy:', test_acc)

if __name__ == "__main__":
    main()
'''

In [ ]:
# Load training and testing sets
train_df = pd.read_csv('train_df_distilbert.csv')
test_df = pd.read_csv('test_df_distilbert.csv')

In [ ]:
# Load the saved embeddings from files
train_embeddings_distil = np.load('train_embeddings_distilbert_1500rows.npy')
test_embeddings_distil = np.load('test_embeddings_distilbert_1500rows.npy')

In [ ]:
# Define the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(768,), 
                       kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.1),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.1),
    keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.1),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the neural network model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# Train the neural network model
model.fit(train_embeddings_distil, train_df['label'], epochs=50, batch_size=32,
          validation_data=(test_embeddings_distil, test_df['label']))

# Evaluate the neural network model on the test data
test_loss, test_acc = model.evaluate(test_embeddings_distil, test_df['label'])
print('Test accuracy:', test_acc)

Epoch 1/50
38/38 [==============================] - 1s 23ms/step - loss: 1.0409 - accuracy: 0.6225 - val_loss: 1.0005 - val_accuracy: 0.7800
Epoch 2/50
38/38 [==============================] - 1s 17ms/step - loss: 0.9677 - accuracy: 0.7508 - val_loss: 0.9165 - val_accuracy: 0.8067
Epoch 3/50
38/38 [==============================] - 1s 16ms/step - loss: 0.8962 - accuracy: 0.7908 - val_loss: 0.8469 - val_accuracy: 0.8300
Epoch 4/50
38/38 [==============================] - 1s 18ms/step - loss: 0.8477 - accuracy: 0.8075 - val_loss: 0.7929 - val_accuracy: 0.8367
Epoch 5/50
38/38 [==============================] - 1s 16ms/step - loss: 0.7852 - accuracy: 0.8300 - val_loss: 0.7503 - val_accuracy: 0.8367
Epoch 6/50
38/38 [==============================] - 1s 18ms/step - loss: 0.7566 - accuracy: 0.8383 - val_loss: 0.7274 - val_accuracy: 0.8433
Epoch 7/50
38/38 [==============================] - 1s 18ms/step - loss: 0.7297 - accuracy: 0.8467 - val_loss: 0.6910 - val_accuracy: 0.8600
Epoch 8/50
38

In [ ]:
# Load training and testing sets
train_df = pd.read_csv('train_df_bert.csv')
test_df = pd.read_csv('test_df_bert.csv')

In [ ]:
# Load the saved embeddings from files
train_embeddings_non_distil = np.load('train_embeddings_bert_1500rows.npy')
test_embeddings_non_distil = np.load('test_embeddings_bert_1500rows.npy')

In [ ]:
# Define the neural network model with L2 regularization and dropout
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(768,), 
                       kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.2),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.2),
    keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the neural network model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# Train the neural network model
model.fit(train_embeddings_non_distil, train_df['label'], epochs=100, batch_size=32,
         validation_data=(test_embeddings_non_distil, test_df['label']))

# Evaluate the neural network model on the test data
test_loss, test_acc = model.evaluate(test_embeddings_non_distil, test_df['label'])
print('Test accuracy:', test_acc)

Epoch 1/100
38/38 [==============================] - 1s 29ms/step - loss: 8.7877 - accuracy: 0.5092 - val_loss: 8.5722 - val_accuracy: 0.5167
Epoch 2/100
38/38 [==============================] - 1s 21ms/step - loss: 8.3998 - accuracy: 0.5700 - val_loss: 8.1915 - val_accuracy: 0.7033
Epoch 3/100
38/38 [==============================] - 1s 20ms/step - loss: 8.0360 - accuracy: 0.6058 - val_loss: 7.8256 - val_accuracy: 0.7467
Epoch 4/100
38/38 [==============================] - 1s 20ms/step - loss: 7.6734 - accuracy: 0.6692 - val_loss: 7.4824 - val_accuracy: 0.7367
Epoch 5/100
38/38 [==============================] - 1s 21ms/step - loss: 7.3473 - accuracy: 0.7000 - val_loss: 7.1588 - val_accuracy: 0.8133
Epoch 6/100
38/38 [==============================] - 1s 21ms/step - loss: 7.0430 - accuracy: 0.7317 - val_loss: 6.8560 - val_accuracy: 0.8600
Epoch 7/100
38/38 [==============================] - 1s 22ms/step - loss: 6.7422 - accuracy: 0.7400 - val_loss: 6.5653 - val_accuracy: 0.8600
Epoch 

38/38 [==============================] - 1s 23ms/step - loss: 1.0994 - accuracy: 0.9125 - val_loss: 1.0847 - val_accuracy: 0.9000
Epoch 59/100
38/38 [==============================] - 1s 21ms/step - loss: 1.0795 - accuracy: 0.9100 - val_loss: 1.0664 - val_accuracy: 0.8767
Epoch 60/100
38/38 [==============================] - 1s 21ms/step - loss: 1.0502 - accuracy: 0.9067 - val_loss: 1.0331 - val_accuracy: 0.9000
Epoch 61/100
38/38 [==============================] - 1s 21ms/step - loss: 1.0185 - accuracy: 0.9067 - val_loss: 1.0093 - val_accuracy: 0.9033
Epoch 62/100
38/38 [==============================] - 1s 22ms/step - loss: 1.0077 - accuracy: 0.9058 - val_loss: 0.9858 - val_accuracy: 0.8967
Epoch 63/100
38/38 [==============================] - 1s 21ms/step - loss: 0.9789 - accuracy: 0.9067 - val_loss: 0.9654 - val_accuracy: 0.8867
Epoch 64/100
38/38 [==============================] - 1s 22ms/step - loss: 0.9575 - accuracy: 0.9075 - val_loss: 0.9445 - val_accuracy: 0.8867
Epoch 65/100